In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import sys

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

if not sys.warnoptions:
    warnings.simplefilter('ignore')

## ETL

In [ ]:
# Somente os dados de treino e isturados (shuffle)
data_train = pd.read_csv("../input/qualityeducation/train.csv", nrows = 1000000).sample(frac=1, random_state=1)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
#Visualizando os dados 
data_train.head()

In [ ]:
data_test = pd.read_csv("../input/qualityeducation/test.csv")

In [ ]:
# Visualizando os dados:
data_test.head()

In [ ]:
data_test_n_inscricoes = data_test['NU_INSCRICAO']
data_test_n_inscricoes.to_csv('n_inscricoes')
train = data_train.drop('NU_INSCRICAO', axis = 1)
test = data_test.drop('NU_INSCRICAO', axis = 1)

# Verificando o tamanho dos datas_sets:
print(len(train))
print(len(test))

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train, test)).reset_index(drop=True)
print("all_data size is : {}".format(all_data.shape))

# Os primeiros milhoes são os dados do dataframe train e o restante é do test...

In [ ]:
# Como eu já tinha feito tudo anteriormente manti os mesmos códigos...
train = all_data

In [ ]:
all_data_na = (train.isnull().sum() / len(train)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
for i in train:
    if train[i].dtypes == 'int64':
        train[i].fillna(value = 0)
    elif train[i].dtypes == 'float64':
        train[i] = train[i].fillna(value = 0) # O problema está aqui: "pd.DataFrame(np.array(train[i]).astype(int))". Pois etá transformando variáveis em numeros negativos.
                                                # E o método que estou usando para transformar isso está demorando demais.
    else:
        train[i].fillna(value = 'None')
        
all_data_na = (train.isnull().sum() / len(train)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
variaveis_problemas = pd.DataFrame(train.isnull().sum()).rename(columns={0: 'NaN'}).query('NaN > 0').T
variaveis_problemas.head()

In [ ]:
for i in variaveis_problemas:
    train[i] = train[i].fillna('None')

In [ ]:
# Verificando se tem algum dado faltante:
all_data_na = (train.isnull().sum() / len(train)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
train_NaN = train.drop(list(train.select_dtypes(include=('int64', 'float64')).columns), axis =1).fillna(0)
NaN = list(train_NaN.select_dtypes(include='object').columns)

for i in NaN:
    print(i, ":", train[i].unique())

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

for i in NaN:
    train[i] = encoder.fit_transform(train_NaN[i])

print(train.dtypes)
print(' ')
train.head()

In [ ]:
# Dados do tipo float está dando um pouco de trabalho...
dados_tipo_float = []

for i in train:
    if train[i].dtypes == 'float64':
        dados_tipo_float.append(i)
    else:
        pass
        
dados_tipo_float

In [ ]:
for i in dados_tipo_float:
    train[i] = np.array(train[i]).astype(int)
    train[i] = train[i].replace(-9223372036854775808, 0)

train.head()

In [ ]:
train = reduce_mem_usage(train)

## Selecionando as melhores variáveis

In [ ]:
# Selecionando as melhores variáveis independetes

participante = train.iloc[:,:18]
escola = train.iloc[:,18:26]
especiais = train.iloc[:,26:77]
local_prova = train.iloc[:,77:81]
dados_prova = train.iloc[:,81:92]
socioecn = train.iloc[:,92:]

In [ ]:
# Separando as variáveis dependentes.

nota_cn = dados_prova.NU_NOTA_CN
nota_ch = dados_prova.NU_NOTA_CH
nota_lc = dados_prova.NU_NOTA_LC
nota_mt = dados_prova.NU_NOTA_MT
nota_reda = dados_prova.NU_NOTA_REDACAO

In [ ]:
# Dropando as notas do dataframe:

dados_prova.drop('NU_NOTA_CN',axis = 1, inplace = True)
dados_prova.drop('NU_NOTA_CH',axis = 1, inplace = True)
dados_prova.drop('NU_NOTA_LC',axis = 1, inplace = True)
dados_prova.drop('NU_NOTA_MT',axis = 1, inplace = True)
dados_prova.drop('NU_NOTA_REDACAO',axis = 1, inplace = True)

In [ ]:
dados_prova

In [ ]:
# Fazer isso por ultimo. O objetivo desse código é apenas separar os dataframes para fazer a modelagem...
print(len(train))

test = train[ntrain:]
train = train[:ntrain]

print(len(test))
print(len(train))

## Selecionando as melhores features

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

def best_features(x_train, y_train):
    clf = ExtraTreesClassifier().fit(x_train, y_train)
    features = pd.DataFrame()
    features['feature']=x_train.columns
    features['importancia'] = clf.feature_importances_
    best_features = features[features['importancia']>np.mean(features['importancia'])].sort_values(by='importancia', ascending = False)[:16].set_index('feature')
    return best_features

## Modelando

In [ ]:
# Pretendo usar decision tree no começo...

# Preciso fazer um código para fazer de maneira automatizada esse dados....